In [1]:
import os, sys
import numpy as np
import plotly.express as px
sys.path.append('../')

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
from onsstove.onsstove import OnSSTOVE
from onsstove.layer import RasterLayer, VectorLayer
from onsstove.raster import interpolate
import time

start = time.time()

## 1. Create an OnSSTOVE model

In [4]:
nepal = OnSSTOVE(project_crs=3857, cell_size=(1000, 1000))
output_directory = 'results'
nepal.output_directory = output_directory

## 2. Read the scenario data

In [5]:
path = r"../EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\OnSSTOVE cases\NP_test_file_social_specs.csv"
nepal.read_scenario_data(path, delimiter=',')

## 3. Read the cooking technologies data

In [6]:
path = r"../EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\OnSSTOVE cases\NP_test_file_tech_specs.csv"
nepal.read_tech_data(path, delimiter=',')

## 4. Add a country mask layer

In [7]:
path = r"../EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Other\Administrative boundaries\Admin lvl 0.shp"
mask_layer = VectorLayer('admin', 'adm_0', layer_path=path)
os.makedirs(f'{output_directory}/admin/adm_0', exist_ok=True)
mask_layer.reproject(3857, f'{output_directory}/admin/adm_0')
nepal.mask_layer = mask_layer
# add admin_1 or 2 or 3 layer to gdf here

## 5. Add a population base layer

In [8]:
path = r"../EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Demand\Population\HRSL\population_npl_2018-10-01_geotiff\population_npl_2018-10-01.tif"
os.makedirs(f'{output_directory}/demographics/population', exist_ok=True)
population = RasterLayer('demographics', 'population', layer_path=path, resample='sum')
population.reproject(3857, f'{output_directory}/demographics/population', 1000, 1000)
population.mask(nepal.mask_layer.layer, f'{output_directory}/demographics/population')
nepal.base_layer = population
nepal.population_to_dataframe(population)

## 6. Calibrate population and urban/rural split

In [9]:
nepal.calibrate_current_pop()
path = r"../EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Other\Urban - Rural divide\GHS.tif"
nepal.calibrate_urban_current_and_future_GHS(path)

## 7. Calculate parameters of base fuel (Biomass)

### 7.1. Health costs

In [10]:
# Calculate relative risk for each disease for the base fuel
rr_0_alri, rr_0_copd, rr_0_ihd, rr_0_lc = nepal.base_fuel.relative_risk()

# Calculate the Population Atributable Factor (PAF) for each disease for the base fuel
paf_0_alri = nepal.base_fuel.paf(rr_0_alri, 1 - nepal.specs['clean_cooking_access'])
paf_0_copd = nepal.base_fuel.paf(rr_0_copd, 1 - nepal.specs['clean_cooking_access'])
paf_0_ihd  = nepal.base_fuel.paf(rr_0_ihd, 1 - nepal.specs['clean_cooking_access'])
paf_0_lc = nepal.base_fuel.paf(rr_0_lc, 1 - nepal.specs['clean_cooking_access'])

### 7.2. Carbon emissions and related costs

In [11]:
nepal.base_fuel.carb(nepal.specs, nepal.gdf)

### 7.3. Time for travelling, collecting fuel, and cooking

In [12]:
# paths to GIS layers
nepal.base_fuel.friction_path = r"../EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Other\Walking friction\2020_walking_only_friction_surface.geotiff"
nepal.base_fuel.forest_path = r"../EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Other\Forest cover\Global Forest Cover Change (GFCC).tif"

# Calculate total time
nepal.base_fuel.total_time(nepal)

## 8. Add wealth index GIS data

In [13]:
path = r"../EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Demand\Wealth Index\Wealth index 2011.tif"
wealth_index = RasterLayer('demographics', 'Wealth_index', layer_path=path,  resample='average')
os.makedirs(f'{output_directory}/demographics/Wealth_index', exist_ok=True)
interpolate(wealth_index.path)
wealth_index.align(population.path, f'{output_directory}/demographics/Wealth_index')

### 8.1. Calculate value of time

In [14]:
# Based on wealth index, minimum wage and a lower an upper range for cost of oportunity
nepal.get_value_of_time(wealth_index)

## 9. Read electricity network GIS layers

In [15]:
# Read MV lines
path = r"../EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Supply\Power network\MV-network\Nepal_DL0.shp"
mv_lines = VectorLayer('electricity', 'MV_line', layer_path=path)
os.makedirs(f'{output_directory}/electricity/MV_line', exist_ok=True)
mv_lines.reproject(3857, f'{output_directory}/electricity/MV_line')

# Read HV lines
path = r"../EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Supply\Power network\HV-network\Existing_transmission_lines.geojson"
hv_lines = VectorLayer('electricity', 'HV_line', layer_path=path)
os.makedirs(f'{output_directory}/electricity/HV_line', exist_ok=True)
hv_lines.reproject(3857, f'{output_directory}/electricity/HV_line')

### 9.1. Calculate distance to electricity infrastructure 

In [16]:
nepal.distance_to_electricity(hv_lines=hv_lines, mv_lines=mv_lines)

## 10. Add night time lights data

In [17]:
path = r"../EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Other\Night Time Lights\nighttime lights.tif"
os.makedirs(f'{output_directory}/electricity/Night_lights', exist_ok=True)
ntl = RasterLayer('electricity', 'Night_lights', layer_path=path, resample='average')
ntl.align(population.path, f'{output_directory}/electricity/Night_lights')
ntl.layer[np.isnan(ntl.layer)] = 0 # hey lets make sure that we interpolate NaNs

nepal.raster_to_dataframe(ntl.layer, name='Night_lights', method='read')

## 11. Calibrate current electrified population

In [18]:
nepal.normalize_for_electricity()
nepal.current_elec()
nepal.final_elec()

print('Calibrated grid electrified population fraction:', nepal.gdf['Elec_pop_calib'].sum() / nepal.gdf['Calibrated_pop'].sum())

Calibrated grid electrified population fraction: 0.7199972171837734


## 12. Adding GIS data for LPG supply

In [19]:
nepal.techs['LPG'].lpg_path = r"../EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Supply\LPG\Nepal_Gas_12Jun2021_Final4.shp"
nepal.techs['LPG'].friction_path = r"../EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Other\Motorized friction\2020_motorized_friction_surface.geotiff"

nepal.techs['LPG'].add_travel_time(population, output_directory)

## 13. Adding GIS data for Improved Biomass collected (ICS biomass)

In [20]:
nepal.techs['Collected_Improved_Biomass'].friction_path = r"../EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Other\Walking friction\2020_walking_only_friction_surface.geotiff"
nepal.techs['Collected_Improved_Biomass'].forest_path = r"../EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Other\Forest cover\Global Forest Cover Change (GFCC).tif"

## 14. Adding GIS data for Improved Biomass collected (ICS biomass)

In [241]:
admin = gpd.read_file(r"../EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Other\Administrative boundaries\Admin lvl 0.shp")
buffaloes = r"../EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Supply\Global livestock\Buffaloes\5_Bf_2010_Da.tif"
cattles = r"../EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Supply\Global livestock\Cattle\5_Ct_2010_Da.tif"
poultry = r"..\EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Supply\Global livestock\Chickens\5_Ch_2010_Da.tif"
goats = r"..\EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Supply\Global livestock\Goats\5_Gt_2010_Da.tif"
pigs = r"..\EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Supply\Global livestock\Pigs\5_Pg_2010_Da.tif"
sheeps = r"..\EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Supply\Global livestock\Sheep\5_Sh_2010_Da.tif"

nepal.techs['Biogas'].recalibrate_livestock(nepal, admin, buffaloes, cattles, poultry, goats, pigs, sheeps)

## 15. Calculate benefits and costs of each technology

In [79]:
names = ['Biogas', 'Electricity', 'Collected_Traditional_Biomass', 'Collected_Improved_Biomass', 'LPG']
techs = [nepal.techs[name] for name in names]

# Loop through each technology and calculate all benefits and costs
for tech in techs:
    print(f'Calculating health benefits for {tech.name}...')
    tech.morbidity(nepal.specs, nepal.gdf, paf_0_alri, paf_0_copd, paf_0_lc, paf_0_ihd)
    tech.mortality(nepal.specs, nepal.gdf, paf_0_alri, paf_0_copd, paf_0_lc, paf_0_ihd)
    print(f'Calculating carbon emissions benefits for {tech.name}...')
    tech.carbon_emissions(nepal.specs, nepal.gdf, nepal.base_fuel.carbon)
    print(f'Calculating time saved benefits for {tech.name}...')
    tech.time_saved(nepal)
    print(f'Calculating costs for {tech.name}...')
    tech.discounted_om(nepal.gdf, nepal.specs)
    tech.discounted_inv(nepal.gdf, nepal.specs)
    tech.discounted_meals(nepal.gdf, nepal.specs)
    tech.discount_fuel_cost(nepal.gdf, nepal.specs, nepal.rows, nepal.cols)
    tech.salvage(nepal.gdf, nepal.specs)
    print(f'Calculating net benefit for {tech.name}...\n')
    tech.net_benefit(nepal.gdf)

Calculating health benefits for Biogas...
Calculating carbon emissions benefits for Biogas...
Calculating time saved benefits for Biogas...
Calculating costs for Biogas...
Calculating net benefit for Biogas...

Calculating health benefits for Electricity...
Calculating carbon emissions benefits for Electricity...
Calculating time saved benefits for Electricity...
Calculating costs for Electricity...
Calculating net benefit for Electricity...

Calculating health benefits for Collected_Traditional_Biomass...
Calculating carbon emissions benefits for Collected_Traditional_Biomass...
Calculating time saved benefits for Collected_Traditional_Biomass...
Calculating costs for Collected_Traditional_Biomass...
Calculating net benefit for Collected_Traditional_Biomass...

Calculating health benefits for Collected_Improved_Biomass...
Calculating carbon emissions benefits for Collected_Improved_Biomass...
Calculating time saved benefits for Collected_Improved_Biomass...
Calculating costs for Colle

## 16. Getting the max benefit technology for each cell

In [80]:
nepal.maximum_net_benefit()

KeyError: 'available_biogas_energy'

## 17. Printing the results

In [ ]:
nepal.gdf

In [ ]:
nepal.lives_saved()
nepal.health_costs_saved()
nepal.extract_time_saved()
nepal.reduced_emissions()
nepal.investment_costs()
nepal.fuel_costs()
nepal.emissions_costs_saved()

In [ ]:
nepal.gdf.groupby(['max_benefit_tech']).agg({'Calibrated_pop': lambda row: np.nansum(row) / 1000000,
                                             'maximum_net_benefit': lambda row: np.nansum(row) / 1000000,
                                             'deaths_avoided': 'sum',
                                             'health_costs_avoided': lambda row: np.nansum(row) / 1000000,
                                             'time_saved': 'sum',
                                             'reduced_emissions': lambda row: np.nansum(row) / 1000000000,
                                             'investment_costs': lambda row: np.nansum(row) / 1000000,
                                             'fuel_costs': lambda row: np.nansum(row) / 1000000,
                                             'emissions_costs_saved': lambda row: np.nansum(row) / 1000000})

In [ ]:
print('maximum_net_benefit:', nepal.gdf['maximum_net_benefit'].sum() / 1000000)
print('deaths_avoided:', nepal.gdf['deaths_avoided'].sum())
print('health_costs_avoided:', nepal.gdf['health_costs_avoided'].sum() / 1000000)
print('time_saved:', nepal.gdf['time_saved'].sum() / nepal.gdf['Households'].sum() / 365)
print('reduced_emissions:', nepal.gdf['reduced_emissions'].sum() / 1000000000)
print('investment_costs:', nepal.gdf['investment_costs'].sum() / 1000000)
print('fuel_costs:', nepal.gdf['fuel_costs'].sum() / 1000000)
print('emissions_costs_saved:', nepal.gdf['emissions_costs_saved'].sum() / 1000000)

## 18. Saving data to raster files

In [ ]:
nepal.to_raster('max_benefit_tech')
nepal.to_raster('net_benefit_Electricity')
nepal.to_raster('net_benefit_LPG')
nepal.to_raster('net_benefit_Collected_Traditional_Biomass')
nepal.to_raster('net_benefit_Collected_Improved_Biomass')
nepal.to_raster('maximum_net_benefit')
nepal.to_raster('investment_costs')

In [ ]:
nepal.gdf.to_csv(os.path.join(output_directory, 'Output', 'sample_output.csv'))

In [ ]:
end = time.time()

diff = end - start
print('Execution time:', str(str(int(diff//60))) + ' min ' + str(int((diff)%60)) + ' sec')

In [ ]:
dff = nepal.gdf.groupby(['max_benefit_tech']).agg({'Calibrated_pop': lambda row: np.nansum(row) / 1000000,
                                                   'maximum_net_benefit': lambda row: np.nansum(row) / 1000000,
                                                   'investment_costs': lambda row: np.nansum(row) / 1000000}).reset_index()

In [ ]:
fig = px.pie(dff, names='max_benefit_tech', values='Calibrated_pop', color_discrete_sequence=['#dc0f0f', '#a86ee1', '#79de13'])
fig.write_image(os.path.join(output_directory, 'output', 'Pop_per_tech.pdf'), width=500, height=500)
fig.update_traces(textfont_size=16)
fig.show()

In [ ]:
fig = px.pie(dff, names='max_benefit_tech', values='investment_costs', color_discrete_sequence=['#dc0f0f', '#a86ee1', '#79de13'])
fig.write_image(os.path.join(output_directory, 'output', 'Investments.pdf'), width=500, height=500)
fig.update_traces(textfont_size=16)
fig.show()